In [12]:
import pandas as pd       
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)
train.shape


(25000, 3)

In [13]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [14]:
print train["sentiment"][1]

1


In [15]:
#On utilise beautifulSoup pour effectuer un premier nettoyage
from bs4 import BeautifulSoup

#On initialise beautifulSoup sur un seul commentaire
example1 = BeautifulSoup(train["review"][1], "lxml") 

#On imprime le commentaire original puis le commentaire nettoyé
#pour obvserver la différence
print train["review"][1]
print example1.get_text()

"\"The Classic War of the Worlds\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \"critics\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \"critics\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \"critics\" perceive to be its shortcomings."
"\"The Classic War of the Worlds\" by Timothy Hines

In [16]:
#On utilise ensuite la librairie re afin d'utiliser des expressions regulières
import re
#On pourra ainsi effectuer des find and replace
letters_only = re.sub("[^a-zA-Z]",            #Le pattern recherché
                      " ",                    #Remplacé par
                      example1.get_text())    #Le texte à manipuler
print letters_only

   The Classic War of the Worlds   by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H  G  Wells  classic book  Mr  Hines succeeds in doing so  I  and those who watched his film with me  appreciated the fact that it was not the standard  predictable Hollywood fare that comes out every year  e g  the Spielberg version with Tom Cruise that had only the slightest resemblance to the book  Obviously  everyone looks for different things in a movie  Those who envision themselves as amateur   critics   look only to criticize everything they can  Others rate a movie on more important bases like being entertained  which is why most people never agree with the   critics    We enjoyed the effort Mr  Hines put into being faithful to H G  Wells  classic novel  and we found it to be very entertaining  This made it easy to overlook what the   critics   perceive to be its shortcomings  


In [17]:
lower_case = letters_only.lower()    #On converti en minuscule
words = lower_case.split()           #On split la review en mots uniques
print lower_case
print words

   the classic war of the worlds   by timothy hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate h  g  wells  classic book  mr  hines succeeds in doing so  i  and those who watched his film with me  appreciated the fact that it was not the standard  predictable hollywood fare that comes out every year  e g  the spielberg version with tom cruise that had only the slightest resemblance to the book  obviously  everyone looks for different things in a movie  those who envision themselves as amateur   critics   look only to criticize everything they can  others rate a movie on more important bases like being entertained  which is why most people never agree with the   critics    we enjoyed the effort mr  hines put into being faithful to h g  wells  classic novel  and we found it to be very entertaining  this made it easy to overlook what the   critics   perceive to be its shortcomings  
[u'the', u'classic', u'war', u'of', u'the', u'world

In [9]:
#On importe nltk et on télécharge les datasets de mots
import nltk
nltk.download()

#On utilisera les "stop-word" mots inutiles pour notre cas
from nltk.corpus import stopwords
#Voici leur liste des mots en anglais :
print stopwords.words("english")

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both'

In [18]:
#On retire les stop words du texte
words = [w for w in words if not w in stopwords.words("english")]
print words

[u'classic', u'war', u'worlds', u'timothy', u'hines', u'entertaining', u'film', u'obviously', u'goes', u'great', u'effort', u'lengths', u'faithfully', u'recreate', u'h', u'g', u'wells', u'classic', u'book', u'mr', u'hines', u'succeeds', u'watched', u'film', u'appreciated', u'fact', u'standard', u'predictable', u'hollywood', u'fare', u'comes', u'every', u'year', u'e', u'g', u'spielberg', u'version', u'tom', u'cruise', u'slightest', u'resemblance', u'book', u'obviously', u'everyone', u'looks', u'different', u'things', u'movie', u'envision', u'amateur', u'critics', u'look', u'criticize', u'everything', u'others', u'rate', u'movie', u'important', u'bases', u'like', u'entertained', u'people', u'never', u'agree', u'critics', u'enjoyed', u'effort', u'mr', u'hines', u'put', u'faithful', u'h', u'g', u'wells', u'classic', u'novel', u'found', u'entertaining', u'made', u'easy', u'overlook', u'critics', u'perceive', u'shortcomings']


In [19]:
def review_to_words(raw_review):
    # Fonciton qui convertit un review en liste de mots
    # L'input est une chaine de caractères,
    # La sortie est une chaine de caractères 
    #
    # 1. On retire le HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text() 
    #
    # 2. On retire toutes les ponctuations        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. On convertit en minuscule puis on tokenize la chaine
    words = letters_only.lower().split()                             
    #
    # 4. On convertir la liste de stop words en set car python 
    #   les traite plus rapidement
    stops = set(stopwords.words("english"))                  
    # 
    # 5. On retire les stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. On retourne le résultat en une seule chaine séparée par un espace.
    return( " ".join( meaningful_words )) 

In [21]:
#Test de la fonction
clean_review = review_to_words( train["review"][1] )
print clean_review

classic war worlds timothy hines entertaining film obviously goes great effort lengths faithfully recreate h g wells classic book mr hines succeeds watched film appreciated fact standard predictable hollywood fare comes every year e g spielberg version tom cruise slightest resemblance book obviously everyone looks different things movie envision amateur critics look criticize everything others rate movie important bases like entertained people never agree critics enjoyed effort mr hines put faithful h g wells classic novel found entertaining made easy overlook critics perceive shortcomings


In [23]:
#On récupère le nombre de review
num_reviews = train["review"].size

#On initialise une liste vide pour contenir tous les reviews
clean_train_reviews = []

#On itère sur chaque review
for i in xrange(0, num_reviews):
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews )
    clean_train_reviews.append(review_to_words(train["review"][i]))

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [25]:
print "Creating the bag of words...\n"
from sklearn.feature_extraction.text import CountVectorizer

#On utilise CountVectorizer, qui est l'outil bag of word de sklearn
#On initialise le max_feature à 5000, ce qui veut dire que notre
#dictionnaire de mot aura une taille maximum de 5000.
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

#fit_transform prépare le modèle, apprend le vocabulaire et transforme 
#nos données en vecteurs.
#L'input est une liste de chaine de caractères.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

#On convertit ensuite le résultat sous forme de tableau
train_data_features = train_data_features.toarray()
print train_data_features.shape


Creating the bag of words...

(25000L, 5000L)


In [28]:
#On affiche le vocabulaire
#vocab = vectorizer.get_feature_names()
#import numpy as np

# Sum up the counts of each vocabulary word
#dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
#for tag, count in zip(vocab, dist):
    #print count, tag

In [29]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


In [31]:
# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

(25000, 2)


In [33]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [35]:
# Get a bag of words for the test set, and convert to a numpy array
print "Creating the bag of words...\n"
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
print train_data_features.shape

Creating the bag of words...

(25000L, 5000L)


In [36]:
# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
print "Done !"

Done !


In [37]:
# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )
print "CSV created !"

CSV created !
